<a href="https://colab.research.google.com/github/Loopercool/Data-Detectives/blob/main/Prashuk_202101211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [ ]:
import math
import numpy as np
import pandas as pd
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_cleaned = pd.read_csv("/content/drive/MyDrive/data_cleaned_energy_included.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
columns_to_drop=['Energy (kWh)','GHG Savings (kg)','Gasoline Savings (gallons)','Currency','Country','Org Name','Plug Type','Port Type','Port Number']
df_main.drop(columns=columns_to_drop, inplace=True)

# **Remove Entries having nan value**

In [ ]:
data_cleaned = df_main.dropna()

In [ ]:
data_cleaned.isnull().sum()

Station Name                       0
MAC Address                        0
Start Date                         0
Start Time Zone                    0
End Date                           0
End Time Zone                      0
Transaction Date (Pacific Time)    0
Total Duration (hh:mm:ss)          0
Charging Time (hh:mm:ss)           0
EVSE ID                            0
Address 1                          0
City                               0
State/Province                     0
Postal Code                        0
Latitude                           0
Longitude                          0
Fee                                0
Ended By                           0
Plug In Event Id                   0
Driver Postal Code                 0
User ID                            0
County                             0
System S/N                         0
Model Number                       0
dtype: int64

In [ ]:
data_cleaned.shape

(169446, 24)

# **Converting *START DATE, Start time Zone* to numeric**

In [ ]:
selected_columns = ["Start Date", "Start Time Zone"]

# Create a new DataFrame with the selected columns
dfa = data_cleaned[selected_columns]

In [ ]:
import pandas as pd
import pytz
from datetime import datetime

# Define time zones
pst = pytz.timezone('US/Pacific')
pdt = pytz.timezone('US/Pacific')

# Function to convert PST to PDT
def convert_pst_to_pdt(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    dt_object = pst.localize(dt_object)
    dt_object_pdt = dt_object.astimezone(pdt)

    return dt_object_pdt

# Function to convert date and time to numeric value
def convert_to_numeric(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    numeric_value = dt_object.timestamp()

    return numeric_value

# Convert PST to PDT and apply numeric conversion
converted_date_times = []
numeric_values = []

for index, row in dfa.iterrows():
    if row["Start Time Zone"] == "PST":
        converted_dt = convert_pst_to_pdt(row["Start Date"])
    else:
        converted_dt = datetime.strptime(row["Start Date"], "%m/%d/%Y %H:%M")

    converted_date_times.append(converted_dt)
    numeric_value = convert_to_numeric(row["Start Date"])
    numeric_values.append(numeric_value)

dfa["ConvertedDateTime"] = converted_date_times
dfa["NumericValue"] = numeric_values

print(dfa)

data_cleaned["Start Date"] = dfa["NumericValue"]

              Start Date Start Time Zone          ConvertedDateTime  \
78948      8/1/2016 5:21             PDT        2016-08-01 05:21:00   
78949      8/1/2016 6:01             PDT        2016-08-01 06:01:00   
78950      8/1/2016 6:03             PDT        2016-08-01 06:03:00   
78951      8/1/2016 6:10             PDT        2016-08-01 06:10:00   
78952      8/1/2016 6:13             PDT        2016-08-01 06:13:00   
...                  ...             ...                        ...   
259409  12/31/2020 15:51             PST  2020-12-31 15:51:00-08:00   
259411  12/31/2020 16:48             PST  2020-12-31 16:48:00-08:00   
259412  12/31/2020 17:28             PST  2020-12-31 17:28:00-08:00   
259413  12/31/2020 17:49             PST  2020-12-31 17:49:00-08:00   
259414  12/31/2020 18:19             PST  2020-12-31 18:19:00-08:00   

        NumericValue  
78948   1.470029e+09  
78949   1.470031e+09  
78950   1.470031e+09  
78951   1.470032e+09  
78952   1.470032e+09  
...      

<ipython-input-8-5fe1e3c85d9c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["ConvertedDateTime"] = converted_date_times
<ipython-input-8-5fe1e3c85d9c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["NumericValue"] = numeric_values
<ipython-input-8-5fe1e3c85d9c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [ ]:
print("Original dataset:")
print(data_cleaned)

# Function to check if a value is a valid date and time format
def is_valid_datetime(value):
    try:
        datetime.strptime(value, '%m/%d/%Y %H:%M')
        return True
    except ValueError:
        return False

# Filter out rows with invalid date and time values
data_cleaned = data_cleaned[df_main['End Date'].apply(is_valid_datetime)]

# Print the dataset after dropping invalid rows
# print("\nDataset after dropping invalid rows:")
# print(df_main)

Original dataset:
                       Station Name          MAC Address    Start Date  \
78948   PALO ALTO CA / CAMBRIDGE #1  0024:B100:0002:1329  1.470029e+09   
78949   PALO ALTO CA / CAMBRIDGE #2  0024:B100:0002:13B7  1.470031e+09   
78950   PALO ALTO CA / CAMBRIDGE #1  0024:B100:0002:1329  1.470031e+09   
78951    PALO ALTO CA / HAMILTON #2  0024:B100:0002:1331  1.470032e+09   
78952        PALO ALTO CA / HIGH #1  0024:B100:0002:10FE  1.470032e+09   
...                             ...                  ...           ...   
259409  PALO ALTO CA / CAMBRIDGE #5  0024:B100:0002:9B7F  1.609430e+09   
259411    PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233  1.609433e+09   
259412        PALO ALTO CA / MPL #4  0024:B100:0003:CD37  1.609436e+09   
259413    PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233  1.609437e+09   
259414  PALO ALTO CA / CAMBRIDGE #4  0024:B100:0003:3A0A  1.609439e+09   

       Start Time Zone          End Date End Time Zone  \
78948              PDT     8/1/2016

<ipython-input-9-80ddd35c4a75>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_cleaned = data_cleaned[df_main['End Date'].apply(is_valid_datetime)]


In [ ]:
selected_columns = ['End Date', 'End Time Zone']

# Create a new DataFrame with the selected columns
dfb = data_cleaned[selected_columns]

In [ ]:
import pandas as pd
import pytz
from datetime import datetime

# Define time zones
pst = pytz.timezone('US/Pacific')
pdt = pytz.timezone('US/Pacific')

# Function to convert PST to PDT
def convert_pst_to_pdt(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    dt_object = pst.localize(dt_object)
    dt_object_pdt = dt_object.astimezone(pdt)

    return dt_object_pdt

# Function to convert date and time to numeric value
def convert_to_numeric(date_str):
    dt_object = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    numeric_value = dt_object.timestamp()

    return numeric_value

# Convert PST to PDT and apply numeric conversion
converted_date_times = []
numeric_values = []

for index, row in dfb.iterrows():
    if row["End Time Zone"] == "PST":
        converted_dt = convert_pst_to_pdt(row["End Date"])
    else:
        converted_dt = datetime.strptime(row["End Date"], "%m/%d/%Y %H:%M")

    converted_date_times.append(converted_dt)
    numeric_value = convert_to_numeric(row["End Date"])
    numeric_values.append(numeric_value)

dfb["ConvertedDateTime"] = converted_date_times
dfb["NumericValue"] = numeric_values

print(dfb)

data_cleaned["End Date"] = dfa["NumericValue"]

                End Date End Time Zone          ConvertedDateTime  \
78948      8/1/2016 6:49           PDT        2016-08-01 06:49:00   
78949     8/1/2016 10:58           PDT        2016-08-01 10:58:00   
78950      8/1/2016 9:15           PDT        2016-08-01 09:15:00   
78951      8/1/2016 8:54           PDT        2016-08-01 08:54:00   
78952      8/1/2016 9:33           PDT        2016-08-01 09:33:00   
...                  ...           ...                        ...   
259409  12/31/2020 17:30           PST  2020-12-31 17:30:00-08:00   
259411  12/31/2020 20:31           PST  2020-12-31 20:31:00-08:00   
259412    1/1/2021 10:56           PST  2021-01-01 10:56:00-08:00   
259413  12/31/2020 21:55           PST  2020-12-31 21:55:00-08:00   
259414  12/31/2020 19:01           PST  2020-12-31 19:01:00-08:00   

        NumericValue  
78948   1.470034e+09  
78949   1.470049e+09  
78950   1.470043e+09  
78951   1.470042e+09  
78952   1.470044e+09  
...              ...  
259409  1.

<ipython-input-11-89e9d7c67890>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb["ConvertedDateTime"] = converted_date_times
<ipython-input-11-89e9d7c67890>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb["NumericValue"] = numeric_values


In [ ]:
data_cleaned.columns

Index(['Station Name', 'MAC Address', 'Start Date', 'Start Time Zone',
       'End Date', 'End Time Zone', 'Transaction Date (Pacific Time)',
       'Total Duration (hh:mm:ss)', 'Charging Time (hh:mm:ss)', 'EVSE ID',
       'Address 1', 'City', 'State/Province', 'Postal Code', 'Latitude',
       'Longitude', 'Fee', 'Ended By', 'Plug In Event Id',
       'Driver Postal Code', 'User ID', 'County', 'System S/N',
       'Model Number'],
      dtype='object')

In [ ]:
columns_to_drop=['Start Time Zone','End Time Zone','Transaction Date (Pacific Time)']
data_cleaned.drop(columns=columns_to_drop, inplace=True)

In [ ]:
def time_to_numeric(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    total_hours = hours + minutes / 60 + seconds / 3600
    return total_hours

data_cleaned['Total_time_hours'] = data_cleaned['Total Duration (hh:mm:ss)'].apply(time_to_numeric)

data_cleaned['Charging_time_hours'] = data_cleaned['Charging Time (hh:mm:ss)'].apply(time_to_numeric)

In [ ]:
columns_to_drop=['Total Duration (hh:mm:ss)','Charging Time (hh:mm:ss)']
data_cleaned.drop(columns=columns_to_drop, inplace=True)

In [ ]:
data_cleaned.columns

Index(['Station Name', 'MAC Address', 'Start Date', 'End Date', 'EVSE ID',
       'Address 1', 'City', 'State/Province', 'Postal Code', 'Latitude',
       'Longitude', 'Fee', 'Ended By', 'Plug In Event Id',
       'Driver Postal Code', 'User ID', 'County', 'System S/N', 'Model Number',
       'Total_time_hours', 'Charging_time_hours'],
      dtype='object')

In [ ]:
data_cleaned.head()

,Station Name,MAC Address,Start Date,End Date,EVSE ID,Address 1,City,State/Province,Postal Code,Latitude,...,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number,Total_time_hours,Charging_time_hours
78948,PALO ALTO CA / CAMBRIDGE #1,0024:B100:0002:1329,1.470029e+09,1.470029e+09,109701.0,400 Cambridge Ave,Palo Alto,California,94306,37.426155,...,0.0,Plug Out at Vehicle,1665,94587.0,523487,Santa Clara County,1.427410e+11,CT4020-HD-GW,1.466111,1.423889
78949,PALO ALTO CA / CAMBRIDGE #2,0024:B100:0002:13B7,1.470031e+09,1.470031e+09,107427.0,400 Cambridge Ave,Palo Alto,California,94306,37.426182,...,0.0,Plug Out at Vehicle,1771,95030.0,141845,Santa Clara County,1.428410e+11,CT4020-HD-GW,4.940833,2.517778
78950,PALO ALTO CA / CAMBRIDGE #1,0024:B100:0002:1329,1.470031e+09,1.470031e+09,109701.0,400 Cambridge Ave,Palo Alto,California,94306,37.426155,...,0.0,Plug Out at Vehicle,1666,80525.0,203603,Santa Clara County,1.427410e+11,CT4020-HD-GW,3.204722,3.188333
78951,PALO ALTO CA / HAMILTON #2,0024:B100:0002:1331,1.470032e+09,1.470032e+09,109785.0,250 Hamilton Ave,Palo Alto,California,94301,37.444359,...,0.0,Plug Out at Vehicle,2093,94539.0,126575,Santa Clara County,1.427410e+11,CT4020-HD-GW,2.740278,2.430278
78952,PALO ALTO CA / HIGH #1,0024:B100:0002:10FE,1.470032e+09,1.470032e+09,104339.0,528 High St,Palo Alto,California,94301,37.443558,...,0.0,Plug Out at Vehicle,3112,94024.0,191743,Santa Clara County,1.422410e+11,CT4020-HD-GW,3.323333,3.320833


# **Label Encode categorical columns**

In [ ]:
categorical_columns = data_cleaned.select_dtypes(include=['object', 'category']).columns

# Print the list of categorical columns
print("Categorical columns:", categorical_columns)

Categorical columns: Index(['Station Name', 'MAC Address', 'Address 1', 'City', 'State/Province',
       'Ended By', 'User ID', 'County', 'Model Number'],
      dtype='object')


In [ ]:
categorical_columns = [col for col in categorical_columns if col != 'User ID']

In [ ]:
print(categorical_columns)

['Station Name', 'MAC Address', 'Address 1', 'City', 'State/Province', 'Ended By', 'County', 'Model Number']


In [ ]:
data_cleaned.drop(['City'],axis=1,inplace = True)

In [ ]:
data_cleaned.drop(['State/Province'],axis=1,inplace=True)

In [ ]:
categorical_columns_1  = ['County', 'Model Number']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for column in categorical_columns:
  le = LabelEncoder()
  data_cleaned[column] = le.fit_transform(data_cleaned[column])
  label_encoders[column] = le

print(data_cleaned)

        Station Name  MAC Address    Start Date      End Date    EVSE ID  \
78948              7           22  1.470029e+09  1.470029e+09   109701.0   
78949              8           24  1.470031e+09  1.470031e+09   107427.0   
78950              7           22  1.470031e+09  1.470031e+09   109701.0   
78951             13           23  1.470032e+09  1.470032e+09   109785.0   
78952             14           17  1.470032e+09  1.470032e+09   104339.0   
...              ...          ...           ...           ...        ...   
259409            11           36  1.609430e+09  1.609430e+09   318651.0   
259411            42           27  1.609433e+09  1.609433e+09   104427.0   
259412            21           57  1.609436e+09  1.609436e+09  2884001.0   
259413            42           27  1.609437e+09  1.609437e+09   104427.0   
259414            10           40  1.609439e+09  1.609439e+09   337571.0   

        Address 1  City  State/Province  Postal Code   Latitude  ...   Fee  \
78948    

In [ ]:
categorical_columns_after_label_encoding = data_cleaned.select_dtypes(include=['object', 'category']).columns

# Print the list of categorical columns
print("Categorical columns:", categorical_columns_after_label_encoding)

Categorical columns: Index(['User ID'], dtype='object')


In [ ]:
unique_count = data_cleaned['Model Number'].nunique()

print("Count of unique entries in 'column_name':", unique_count)

print(data_cleaned['Model Number'])

Count of unique entries in 'column_name': 10
78948     6
78949     6
78950     6
78951     6
78952     6
         ..
259409    5
259411    6
259412    4
259413    6
259414    6
Name: Model Number, Length: 169408, dtype: int64


In [ ]:
data_cleaned.head()

,Station Name,MAC Address,Start Date,End Date,EVSE ID,Address 1,Postal Code,Latitude,Longitude,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number,Total_time_hours,Charging_time_hours
78948,7,22,1.470029e+09,1.470029e+09,109701.0,6,94306,37.426155,-122.146065,0.0,8,1665,94587.0,523487,1,1.427410e+11,6,1.466111,1.423889
78949,8,24,1.470031e+09,1.470031e+09,107427.0,6,94306,37.426182,-122.146065,0.0,8,1771,95030.0,141845,1,1.428410e+11,6,4.940833,2.517778
78950,7,22,1.470031e+09,1.470031e+09,109701.0,6,94306,37.426155,-122.146065,0.0,8,1666,80525.0,203603,1,1.427410e+11,6,3.204722,3.188333
78951,13,23,1.470032e+09,1.470032e+09,109785.0,1,94301,37.444359,-122.159904,0.0,8,2093,94539.0,126575,1,1.427410e+11,6,2.740278,2.430278
78952,14,17,1.470032e+09,1.470032e+09,104339.0,11,94301,37.443558,-122.162849,0.0,8,3112,94024.0,191743,1,1.422410e+11,6,3.323333,3.320833


# **Save CSV to google drive**

In [ ]:
path = '/content/drive/MyDrive/Data_Mining/data_cleaned.csv'

data_cleaned.to_csv(path,index=False)

In [ ]:
data_cleaned = pd.read_csv('/content/drive/MyDrive/Data_Mining/data_cleaned.csv')

# **Regression**

In [ ]:
data_df = data_cleaned.drop(['Charging_time_hours'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn import preprocessing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_df, data_cleaned['Charging_time_hours'], random_state=42, test_size=0.20)

In [ ]:
size_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = size_scaler.transform(X_train)
X_test_scaled = size_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((135526, 19), (33882, 19))

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor

# **Linear Regression**

In [ ]:
model_linear_regression = LinearRegression()

model_linear_regression.fit(X_train_scaled, y_train)

predicted_values_linear_reg = model_linear_regression.predict(X_test_scaled)

print("Coefficients of liner regression:", model_linear_regression.coef_)
print("Intercept of liner regression:", model_linear_regression.intercept_)

linear_mse = mean_squared_error(y_test, predicted_values_linear_reg)
linear_mae = mean_absolute_error(y_test, predicted_values_linear_reg)
linear_r2 = r2_score(y_test, predicted_values_linear_reg)

print("Linear Regression MSE:", linear_mse)
print("Linear Regression MAE:", linear_mae)
print("Linear Regression R-squared:", linear_r2)

Coefficients of liner regression: [-0.01125326  0.09586783 -0.01049904 -0.01049904  1.00058334 -0.0245372
 -0.04252808  0.03055208  0.01257465  0.01167461 -0.07173291 -0.00714017
  0.00298375  0.01552216 -0.01108705  0.07561871 -0.09337022 -0.00953799
  0.39572436]
Intercept of liner regression: 1.96468608729439
Linear Regression MSE: 0.250024402771547
Linear Regression MAE: 0.3745787101578202
Linear Regression R-squared: 0.8655376625160813


## **Cross Fold Validation**

In [ ]:
X = data_df
y = data_cleaned['Charging_time_hours']

size_scaler = preprocessing.StandardScaler().fit(X)
X_train_scaled_cross = size_scaler.transform(X)

In [ ]:
model = LinearRegression()

# Perform cross-validation (e.g., 5-fold cross-validation)
scores = cross_val_score(model, X_train_scaled_cross, y, cv=10, scoring='neg_mean_squared_error')

# Convert the scores to positive values (MSE is negative by default)
mse_scores = -scores

# Calculate the mean and standard deviation of the MSE scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

print(f'mean_mse : {mean_mse}')
print(f'std_mse : {std_mse}')

mean_mse : 0.2671929179323745
std_mse : 0.05295095767318782


# **SGDRegression**

In [ ]:
model_sgdregression = SGDRegressor(max_iter=100000, learning_rate='constant', eta0=0.0001, random_state=42)

model_sgdregression.fit(X_train_scaled, y_train)

predicted_values_sgd_regression = model_sgdregression.predict(X_test_scaled)

print("Coefficients of sdg regression:", model_sgdregression.coef_)
print("Intercept of sdg regression:", model_sgdregression.intercept_)

sgd_mse = mean_squared_error(y_test, predicted_values_sgd_regression)
sgd_mae = mean_absolute_error(y_test, predicted_values_sgd_regression)
sgd_r2 = r2_score(y_test, predicted_values_sgd_regression)

print("SGD Regression MSE:", sgd_mse)
print("SGD Regression MAE:", sgd_mae)
print("SGD Regression R-squared:", sgd_r2)

Coefficients of sdg regression: [-0.00949933  0.09055095 -0.0131234  -0.0131234   1.00999657 -0.02114983
 -0.04669521  0.02982999  0.01025199  0.00519987 -0.07485407 -0.0074524
  0.00237312  0.0143155  -0.01352669  0.07659481 -0.09090635 -0.0057149
  0.39098086]
Intercept of sdg regression: [1.96337166]
SGD Regression MSE: 0.25055285639463104
SGD Regression MAE: 0.3748570130196398
SGD Regression R-squared: 0.8652534618195731


## **Cross fold validation**

In [ ]:
model_sgdregression = SGDRegressor(max_iter=100000, learning_rate='constant', eta0=0.0001, random_state=42)

# Perform cross-validation (e.g., 5-fold cross-validation)
scores = cross_val_score(model_sgdregression, X_train_scaled_cross, y, cv=20, scoring='neg_mean_squared_error')

# Convert the scores to positive values (MSE is negative by default)
mse_scores = -scores

# Calculate the mean and standard deviation of the MSE scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

print(f'mean_mse : {mean_mse}')
print(f'std_mse : {std_mse}')

mean_mse : 0.2635245064084825
std_mse : 0.07482219283737619
